In [ ]:
from dolfin import *
from ufl_legacy import le
import operator
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

# Spatial Refinements

In [ ]:
# Q-Tensor parameters
n = 2
num_steps = 1000
T = 0.4
dt = T/num_steps
a = -0.3
b = -4.
c = 4.
M = 1.
L = 1.

# Parameters for electric field
R = 2.
eps2 = 0.5
eps3 = 0.01
eps1 = 2.5

# Parameters for convex splitting
beta1 = 8.
beta2 = 8.

In [ ]:
# Create mesh
mesh = RectangleMesh(Point(-0.5, -0.5), Point(0.5, 0.5), 300, 300)# 200, 200)

In [ ]:
# some functions for the scheme
I = Constant(np.eye(n)) # identity matrix
Zero = 0 * I
Zero_func = 0*Function(FunctionSpace(mesh, "Lagrange", 1))

def dF1_dQ(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    result = beta1 * Q - b*(Q2 - 1/n * trQ2 * I) + beta2 * trQ2 * Q
    return result

def dF2_dQ(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    result = (beta1 - a) * Q + (beta2 - c) * trQ2 * Q
    return result

def F_tot(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    trQ3 = tr(Q2*Q)
    result = a/2 * trQ2 - b/3 * trQ3 + c/4 * trQ2**2
    return result

def tens(Q):
    """
    Make vector function into tensor
    """
    return as_tensor([[Q[0], Q[1]],
                      [Q[1], -1*Q[0]]])

def tens_hi(Q):
    return as_tensor([[Q[0], Q[1]],
                      [Zero_func, Zero_func]])

def tens_lo(Q):
    return as_tensor([[Zero_func, Zero_func],
                      [Q[0], Q[1]]])

# approximate Heaviside function
steepness = 5

def H_(x):
    return 1/np.pi * atan(steepness * x) + 0.5

def DH_(x):
    return 1/np.pi * steepness / (1 + (steepness * x)**2)

# truncation of one component to R/d, d=2
def TR_1D(x):
    return x * H_(R/2-x) * H_(R/2+x) + R/2*H_(x-R/2) - R/2*H_(-x-R/2)

# derivative of TR_1D
def calP_1D(x):
    return H_(R/2-x)*H_(R/2+x) - x*DH_(R/2-x)*H_(R/2+x) + x*H_(R/2-x)*DH_(R/2+x) + R/2*DH_(x-R/2) + R/2*DH_(-x-R/2)

# now if Q is a tensor
def TR(Q):
    return as_tensor([[TR_1D(Q[0, 0]), TR_1D(Q[0, 1])], 
                      [TR_1D(Q[1, 0]), TR_1D(Q[1, 1])]])

def calP(Q):
    return as_tensor([[calP_1D(Q[0, 0]), calP_1D(Q[0, 1])], 
                      [calP_1D(Q[1, 0]), calP_1D(Q[1, 1])]])

def calP_h(Q1, Q0):
    P11 = calP_1D(Q0[0, 0])
    P12 = calP_1D(Q0[0, 1])
    P21 = calP_1D(Q0[1, 0])
    P22 = calP_1D(Q0[1, 1])
    
    res = as_tensor([[P11, P12],
                      [P21, P22]])
    return res

def element_mult(A, B):
    return as_tensor([[A[0, 0] * B[0, 0], A[0, 1] * B[0, 1]], 
                      [A[1, 0] * B[1, 0], A[1, 1] * B[1, 1]]])

### Initialize $Q_h^0$ and $u_h^0$

In [ ]:
# initial conditions for the Q tensor
class InitialConditions(UserExpression):
    def eval(self, values, x):
        x, y = x[0], x[1]
        if np.isclose(x, -0.5) or np.isclose(x, 0.5) or np.isclose(y, -0.5) or np.isclose(y, 0.5):
            values[0] = 0.0
            values[1] = 0.0
        else:
            n0 = (x+0.5)*(x-0.5)*(y+0.5)*(y-0.5)
            n1 = (x+0.5)*(x-0.5)*(y+0.5)*(y-0.5)
            d = np.array([[n0], [n1]]) # director
            Q0 = d @ d.T - np.sum(d*d)/2.0 * np.eye(2) # Q tensor

            values[0] = Q0[0, 0]
            values[1] = Q0[0, 1]
    def value_shape(self):
        return (2,)

In [ ]:
# initialize Q tensor at time 0
Q0 = Function(VectorFunctionSpace(mesh, "Lagrange", 1, dim=2))
Q_init = InitialConditions()
Q0.interpolate(Q_init)

# define uninitialized u0 and linear form to solve for u0
u_space = FunctionSpace(mesh, "Lagrange", 1)
u0 = Function(u_space)
v = TestFunction(u_space)

L_elliptic = inner(grad(u0) + eps2*TR(tens(Q0))*grad(u0) + eps3*div(tens(Q0)), grad(v))*dx

# define boundary condition for u0
t = 0.0
u_b = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])', degree=2)
bc_u = DirichletBC(u_space, u_b, DomainBoundary())

# solve for u0
solve(L_elliptic == 0, u0, bc_u)

## Compute Reference Solution

In [ ]:
class QTensorEquation(NonlinearProblem):
    def __init__(self, L, a, bcs):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
        self.bcs = bcs

    def F(self, b, x):
        assemble(self.L, tensor=b)
        for bc in self.bcs:
            bc.apply(b, x)

    def J(self, A, x):
        assemble(self.a, tensor=A)
        for bc in self.bcs:
            bc.apply(A)

In [ ]:
Q_space = VectorElement("Lagrange", mesh.ufl_cell(), 1, dim=2)
u_space = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
ME = MixedElement(Q_space, u_space)
V = FunctionSpace(mesh, ME)

In [ ]:
# Define trial and test functions
dQu = TrialFunction(V)
q, v = TestFunctions(V)
r, _ = TestFunctions(V)

# Define functions
Qu = Function(V)

# Split mixed functions
dQ, du = split(dQu)
Q, u = split(Qu)

In [ ]:
# Weak statement of the equation
L_form = inner(tens(Q - Q0), tens(q))*dx\
    + M*L*dt/2*inner(grad(tens(Q+Q0)), grad(tens(q)))*dx\
    + M*dt*inner(dF1_dQ(tens(Q)), tens(q))*dx\
    - M*dt*inner(dF2_dQ(tens(Q0)), tens(q))*dx\
    - M*eps2*dt/2*(inner(element_mult(calP_h(tens(Q), tens(Q0)), outer(grad(u0), grad(u))) - (1/n)*tr(element_mult(calP_h(tens(Q), tens(Q0)), outer(grad(u0), grad(u))))*I, tens(q))*dx)\
    - M*eps3*dt/2*(inner(grad(u+u0), div(sym(tens_hi(q))))*dx - (1/n)*inner(grad(u+u0), grad(tr(tens(q))))*dx)\
    + dt*inner(grad(u) + eps2*TR(tens(Q))*grad(u) + eps3*div(tens(Q)), grad(v))*dx

a_form = derivative(L_form, Qu, dQu)

In [ ]:
# boundary conditions for Q do not change
bc_Q = DirichletBC(V.sub(0), np.zeros(2), DomainBoundary())

# Store solutions and times
Q_array = Q0.vector()[:].reshape(-1, 2)
Q_11 = Q_array[:, 0]
Q_12 = Q_array[:, 1]
times = [0.0]

u_b_expr = Expression('10*sin(omega*t + phase)*(x[0]+0.5)*sin(pi*x[1])',
                      degree=2, omega=2*np.pi, t=0.0, phase=0.2, pi=np.pi)
bc_u = DirichletBC(V.sub(1), u_b_expr, DomainBoundary())

problem = QTensorEquation(L_form, a_form, [bc_Q, bc_u])
solver = NewtonSolver()
solver.parameters["linear_solver"] = "gmres"
solver.parameters["preconditioner"] = "ilu"

assigner_Q = FunctionAssigner(Q0.function_space(), V.sub(0))
assigner_u = FunctionAssigner(u0.function_space(), V.sub(1))

t = 0.0
while (t < T):
    print(t)
    t += dt
    times.append(t)
    
    u_b_expr.t = t
    
    solver.solve(problem, Qu.vector())

    assigner_Q.assign(Q0, Qu.sub(0))
    assigner_u.assign(u0, Qu.sub(1))



## Now errors compared to reference solution

In [ ]:
# Q-Tensor parameters
num_steps = 100
T = 0.4
dt = T/num_steps

In [ ]:
mesh_sizes = []
Q_L2_errors = []
Q_H1_errors = []
u_L2_errors = []
u_H1_errors = []

for num_grid_points in [10, 20, 35, 50]:
    # Create mesh
    mesh1 = RectangleMesh(Point(-0.5, -0.5), Point(0.5, 0.5), num_grid_points, num_grid_points)
    mesh_sizes.append(1/num_grid_points)

    # initialize Q tensor at time 0
    Q1 = Function(VectorFunctionSpace(mesh1, "Lagrange", 1, dim=2))
    Q_init = InitialConditions()
    Q1.interpolate(Q_init)
    
    # define uninitialized u0 and linear form to solve for u0
    u_space = FunctionSpace(mesh1, "Lagrange", 1)
    u1 = Function(u_space)
    v = TestFunction(u_space)
    
    L_elliptic = inner(grad(u1) + eps2*TR(tens(Q1))*grad(u1) + eps3*div(tens(Q1)), grad(v))*dx
    
    # define boundary condition for u0
    t = 0.0
    u_b = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])', degree=2)
    bc_u = DirichletBC(u_space, u_b, DomainBoundary())
    
    # solve for u0
    solve(L_elliptic == 0, u1, bc_u)

    Q_space1 = VectorElement("Lagrange", mesh1.ufl_cell(), 1, dim=2)
    u_space1 = FiniteElement("Lagrange", mesh1.ufl_cell(), 1)
    ME1 = MixedElement(Q_space1, u_space1)
    V1 = FunctionSpace(mesh1, ME1)

    # Define trial and test functions
    dQu = TrialFunction(V1)
    q, v = TestFunctions(V1)
    r, _ = TestFunctions(V1)
    
    # Define functions
    Qu = Function(V1)
    
    # Split mixed functions
    dQ, du = split(dQu)
    Q, u = split(Qu)

    # Weak statement of the equation
    L_form = inner(tens(Q - Q1), tens(q))*dx\
        + M*L*dt/2*inner(grad(tens(Q+Q1)), grad(tens(q)))*dx\
        + M*dt*inner(dF1_dQ(tens(Q)), tens(q))*dx\
        - M*dt*inner(dF2_dQ(tens(Q1)), tens(q))*dx\
        - M*eps2*dt/2*(inner(element_mult(calP_h(tens(Q), tens(Q1)), outer(grad(u1), grad(u))) - (1/n)*tr(element_mult(calP_h(tens(Q), tens(Q1)), outer(grad(u1), grad(u))))*I, tens(q))*dx)\
        - M*eps3*dt/2*(inner(grad(u+u1), div(sym(tens_hi(q))))*dx - (1/n)*inner(grad(u+u1), grad(tr(tens(q))))*dx)\
        + dt*inner(grad(u) + eps2*TR(tens(Q))*grad(u) + eps3*div(tens(Q)), grad(v))*dx
    
    a_form = derivative(L_form, Qu, dQu)

    # boundary conditions for Q do not change
    bc_Q = DirichletBC(V1.sub(0), np.zeros(2), DomainBoundary())
    
    # Store solutions and times
    Q_array = Q1.vector()[:].reshape(-1, 2)
    Q_11 = Q_array[:, 0]
    Q_12 = Q_array[:, 1]
    solutions_Q_1 = [np.array([[Q_11, Q_12], [Q_12, -Q_11]]).transpose(2, 0, 1)]
    solutions_u_1 = [np.array(u0.vector()[:].reshape(-1, 1))]
    times_1 = [0.0]
    
    solver = NewtonSolver()
    
    t = 0.0
    while (t < T):
        print(t)
        t += dt
        times_1.append(t)
        
        u_b = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])', degree=2)
        bc_u = DirichletBC(V1.sub(1), u_b, DomainBoundary())
        
        problem = QTensorEquation(L_form, a_form, [bc_Q, bc_u])
        solver.solve(problem, Qu.vector())
        
        Q_array = Qu.sub(0, deepcopy=True).vector()[:].reshape(-1, 2)
        Q_11 = Q_array[:, 0]
        Q_12 = Q_array[:, 1]
        
        solutions_Q_1.append(np.array([[Q_11, Q_12], [Q_12, -Q_11]]).transpose(2, 0, 1))
        solutions_u_1.append(np.array(Qu.sub(1, deepcopy=True).vector()[:].reshape(-1, 1)))
        
        Q1.vector()[:] = np.ascontiguousarray(Qu.sub(0, deepcopy=True).vector()[:])
        u1.vector()[:] = np.ascontiguousarray(Qu.sub(1, deepcopy=True).vector()[:])

    Q_L2_errors.append(errornorm(Q0, Q1, 'L2', degree_rise=0, mesh=mesh))
    Q_H1_errors.append(errornorm(Q0, Q1, 'H1', degree_rise=0, mesh=mesh))
    u_L2_errors.append(errornorm(u0, u1, 'L2', degree_rise=0, mesh=mesh))
    u_H1_errors.append(errornorm(u0, u1, 'H1', degree_rise=0, mesh=mesh))

In [ ]:
print(mesh_sizes, Q_L2_errors, Q_H1_errors, u_L2_errors, u_H1_errors)

In [ ]:
plt.loglog(mesh_sizes, Q_H1_errors)
plt.scatter(mesh_sizes, Q_H1_errors)
plt.loglog(mesh_sizes, mesh_sizes)
plt.show()

In [ ]:
plt.loglog(mesh_sizes, u_H1_errors)
plt.scatter(mesh_sizes, u_H1_errors)
plt.loglog(mesh_sizes, mesh_sizes)
plt.show()

In [ ]:
# Q H1 errors
(np.log(np.array(Q_H1_errors)[1:]) - np.log(np.array(Q_H1_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# Q L2 errors
(np.log(np.array(Q_L2_errors)[1:]) - np.log(np.array(Q_L2_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# u H1 errors
(np.log(np.array(u_H1_errors)[1:]) - np.log(np.array(u_H1_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# u L2 errors
(np.log(np.array(u_L2_errors)[1:]) - np.log(np.array(u_L2_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

# Time refinement

In [ ]:
# Compute reference solution

num_grid_points = 100
num_steps = 8000

T = 0.2
dt = T/num_steps

# Create mesh
mesh2 = RectangleMesh(Point(-0.5, -0.5), Point(0.5, 0.5), num_grid_points, num_grid_points)
#mesh_sizes.append(dt)

# initialize Q tensor at time 0
Q2 = Function(VectorFunctionSpace(mesh2, "Lagrange", 1, dim=2))
Q_init = InitialConditions()
Q2.interpolate(Q_init)

# define uninitialized u0 and linear form to solve for u0
u_space = FunctionSpace(mesh2, "Lagrange", 1)
u2 = Function(u_space)
v = TestFunction(u_space)

L_elliptic = inner(grad(u2) + eps2*TR(tens(Q2))*grad(u2) + eps3*div(tens(Q2)), grad(v))*dx

# define boundary condition for u0
t = 0.0
u_b = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])', degree=6)
bc_u = DirichletBC(u_space, u_b, DomainBoundary())

# solve for u0
solve(L_elliptic == 0, u2, bc_u)

Q_space2 = VectorElement("Lagrange", mesh2.ufl_cell(), 1, dim=2)
u_space2 = FiniteElement("Lagrange", mesh2.ufl_cell(), 1)
ME2 = MixedElement(Q_space2, u_space2)
V2 = FunctionSpace(mesh2, ME2)

# Define trial and test functions
dQu = TrialFunction(V2)
q, v = TestFunctions(V2)
r, _ = TestFunctions(V2)

# Define functions
Qu = Function(V2)

# Split mixed functions
dQ, du = split(dQu)
Q, u = split(Qu)

# Weak statement of the equation
L_form = inner(tens(Q - Q2), tens(q))*dx\
    + M*L*dt/2*inner(grad(tens(Q+Q2)), grad(tens(q)))*dx\
    + M*dt*inner(dF1_dQ(tens(Q)), tens(q))*dx\
    - M*dt*inner(dF2_dQ(tens(Q2)), tens(q))*dx\
    - M*eps2*dt/2*(inner(element_mult(calP_h(tens(Q), tens(Q2)), outer(grad(u2), grad(u))) - (1/n)*tr(element_mult(calP_h(tens(Q), tens(Q2)), outer(grad(u2), grad(u))))*I, tens(q))*dx)\
    - M*eps3*dt/2*(inner(grad(u+u2), div(sym(tens_hi(q))))*dx - (1/n)*inner(grad(u+u2), grad(tr(tens(q))))*dx)\
    + dt*inner(grad(u) + eps2*TR(tens(Q))*grad(u) + eps3*div(tens(Q)), grad(v))*dx

a_form = derivative(L_form, Qu, dQu)

# boundary conditions for Q do not change
bc_Q = DirichletBC(V2.sub(0), np.zeros(2), DomainBoundary())

# Store solutions and times
times_2 = [0.0]

u_b_expr = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])',
                      degree=6, t=0.0)
bc_u = DirichletBC(V2.sub(1), u_b_expr, DomainBoundary())

problem = QTensorEquation(L_form, a_form, [bc_Q, bc_u])
solver = NewtonSolver()
solver.parameters["linear_solver"] = "gmres"
solver.parameters["preconditioner"] = "ilu"

assigner_Q = FunctionAssigner(Q2.function_space(), V2.sub(0))
assigner_u = FunctionAssigner(u2.function_space(), V2.sub(1))

t = 0.0
while (t < T):
    print(t)
    t += dt
    times_2.append(t)
    
    u_b_expr.t = t
    
    solver.solve(problem, Qu.vector())
    
    assigner_Q.assign(Q2, Qu.sub(0))
    assigner_u.assign(u2, Qu.sub(1))

In [ ]:
mesh_sizes = []
Q_L2_errors = []
Q_H1_errors = []
u_L2_errors = []
u_H1_errors = []

num_grid_points = 100#150

for num_steps in [40, 80, 160, 320, 640, 1280]:
    T = 0.2
    dt = T/num_steps
    
    # Create mesh
    mesh1 = RectangleMesh(Point(-0.5, -0.5), Point(0.5, 0.5), num_grid_points, num_grid_points)
    mesh_sizes.append(dt)

    # initialize Q tensor at time 0
    Q1 = Function(VectorFunctionSpace(mesh1, "Lagrange", 1, dim=2))
    Q_init = InitialConditions()
    Q1.interpolate(Q_init)
    
    # define uninitialized u0 and linear form to solve for u0
    u_space = FunctionSpace(mesh1, "Lagrange", 1)
    u1 = Function(u_space)
    v = TestFunction(u_space)
    
    L_elliptic = inner(grad(u1) + eps2*TR(tens(Q1))*grad(u1) + eps3*div(tens(Q1)), grad(v))*dx
    
    # define boundary condition for u0
    t = 0.0
    u_b = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])', degree=6)
    bc_u = DirichletBC(u_space, u_b, DomainBoundary())
    
    # solve for u0
    solve(L_elliptic == 0, u1, bc_u)

    Q_space1 = VectorElement("Lagrange", mesh1.ufl_cell(), 1, dim=2)
    u_space1 = FiniteElement("Lagrange", mesh1.ufl_cell(), 1)
    ME1 = MixedElement(Q_space1, u_space1)
    V1 = FunctionSpace(mesh1, ME1)

    # Define trial and test functions
    dQu = TrialFunction(V1)
    q, v = TestFunctions(V1)
    r, _ = TestFunctions(V1)
    
    # Define functions
    Qu = Function(V1)
    
    # Split mixed functions
    dQ, du = split(dQu)
    Q, u = split(Qu)

    # Weak statement of the equation
    L_form = inner(tens(Q - Q1), tens(q))*dx\
        + M*L*dt/2*inner(grad(tens(Q+Q1)), grad(tens(q)))*dx\
        + M*dt*inner(dF1_dQ(tens(Q)), tens(q))*dx\
        - M*dt*inner(dF2_dQ(tens(Q1)), tens(q))*dx\
        - M*eps2*dt/2*(inner(element_mult(calP_h(tens(Q), tens(Q1)), outer(grad(u1), grad(u))) - (1/n)*tr(element_mult(calP_h(tens(Q), tens(Q1)), outer(grad(u1), grad(u))))*I, tens(q))*dx)\
        - M*eps3*dt/2*(inner(grad(u+u1), div(sym(tens_hi(q))))*dx - (1/n)*inner(grad(u+u1), grad(tr(tens(q))))*dx)\
        + dt*inner(grad(u) + eps2*TR(tens(Q))*grad(u) + eps3*div(tens(Q)), grad(v))*dx
    
    a_form = derivative(L_form, Qu, dQu)

    # boundary conditions for Q do not change
    bc_Q = DirichletBC(V1.sub(0), np.zeros(2), DomainBoundary())
    
    # Store solutions and times
    times_1 = [0.0]
    
    u_b_expr = Expression(f'10*sin({2*np.pi*t+0.2})*(x[0]+0.5)*sin({np.pi}*x[1])',
                      degree=6, t=0.0)
    bc_u = DirichletBC(V1.sub(1), u_b_expr, DomainBoundary())
    
    problem = QTensorEquation(L_form, a_form, [bc_Q, bc_u])
    solver = NewtonSolver()
    solver.parameters["linear_solver"] = "gmres"
    solver.parameters["preconditioner"] = "ilu"
    
    assigner_Q = FunctionAssigner(Q1.function_space(), V1.sub(0))
    assigner_u = FunctionAssigner(u1.function_space(), V1.sub(1))
    
    t = 0.0
    while (t < T):
        print(t)
        t += dt
        times_1.append(t)
        
        u_b_expr.t = t
    
        solver.solve(problem, Qu.vector())
        
        assigner_Q.assign(Q1, Qu.sub(0))
        assigner_u.assign(u1, Qu.sub(1))

    Q_L2_errors.append(errornorm(Q2, Q1, 'L2', mesh=mesh2))
    Q_H1_errors.append(errornorm(Q2, Q1, 'H1', mesh=mesh2))
    u_L2_errors.append(errornorm(u2, u1, 'L2', mesh=mesh2))
    u_H1_errors.append(errornorm(u2, u1, 'H1', mesh=mesh2))

In [ ]:
print(mesh_sizes, Q_L2_errors, Q_H1_errors, u_L2_errors, u_H1_errors)

In [ ]:
plt.loglog(mesh_sizes, Q_H1_errors)
plt.scatter(mesh_sizes, Q_H1_errors)
plt.loglog(mesh_sizes, np.array(mesh_sizes))
plt.show()

In [ ]:
plt.loglog(mesh_sizes, u_H1_errors)
plt.scatter(mesh_sizes, u_H1_errors)
plt.loglog(mesh_sizes, mesh_sizes)
plt.show()

In [ ]:
# Q H1 errors
(np.log(np.array(Q_H1_errors)[1:]) - np.log(np.array(Q_H1_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# Q L2 errors
(np.log(np.array(Q_L2_errors)[1:]) - np.log(np.array(Q_L2_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# u H1 errors
(np.log(np.array(u_H1_errors)[1:]) - np.log(np.array(u_H1_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])

In [ ]:
# u L2 errors
(np.log(np.array(u_L2_errors)[1:]) - np.log(np.array(u_L2_errors)[:-1])) / (np.log(np.array(mesh_sizes)[1:]) - np.log(np.array(mesh_sizes))[:-1])